# Update training data with manually drawn polygons

This notebook will merge manually drawn crop/non-crop polygons (done either QGIS or ArcGIS) with the training data collected using Collect Earth.

During each iteration of this procedure, update the suffix of the output file with the date of creation in format YYYYMMDD, this will help keep track of which iteration of training data is used for which set of classifications.

***

*Filename guide:*

* `<aez>_training_data_<YYYYMMDD>.geojson`: The training dataset that includes CEO data, manually collected polygons, and any pre-existing datasets.
* `ceo_td_polys.geojson` : training data polygons retrievd from Collect Earth, these are combined the manually collected polygons and any pre-existing datasets to produce the `<aez>_training_data_<date_of_creation>.geojson` file


In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

## Analysis Parameters

In [2]:
date_suffix='20220225' # keep date suffix for all code run using this training data

original_td = 'data/ceo_td_polys.geojson' #shouldn't need to change this

updated_td = 'data/central_manual_crop_polys.shp' #the file you've been adding new TD polygons too in GIS.

## Open vector files

In [3]:
#add manually collected polygons
manual = gpd.read_file(updated_td)
ceo = gpd.read_file(original_td)

## Reclassify Class field

In [4]:
manual['Class'] = np.where(manual['Class']=='crop', 1, manual['Class'])
manual['Class'] = np.where(manual['Class']=='non-crop', 0, manual['Class'])

## Merge files together

In [5]:
training_data = pd.concat([manual,ceo]).reset_index(drop=True)

## Ensure class is in integer type

In [6]:
training_data['Class'] = training_data['Class'].astype(int)

## Counts for each class

In [7]:
print('No. of samples: '+str(len(training_data)))
print('Crop samples = '+str(len(training_data[training_data['Class']==1])))
print('Non-Crop samples = '+str(len(training_data[training_data['Class']==0])))

No. of samples: 1745
Crop samples = 678
Non-Crop samples = 1067


## Export to disk

This file will be the new training data to pass into the `1_Extract_training_data.ipynb` notebook

In [8]:
training_data.to_file('data/Central_training_data_'+date_suffix+'.geojson', driver='GeoJSON')